In [ ]:
! gdown https://drive.google.com/u/0/uc?id=17WpSslT1bv81vHy3k1u0lo53_S10evDe
# For the train+val set
! unzip -qq CXR_Covid-19_Challenge.zip

Downloading...
From: https://drive.google.com/u/0/uc?id=17WpSslT1bv81vHy3k1u0lo53_S10evDe
To: /content/CXR_Covid-19_Challenge.zip
3.61GB [00:56, 63.9MB/s]


In [ ]:
# For the test data
! gdown https://drive.google.com/u/0/uc?id=13S_GWX1f6K0ySqgeuQmH0pShhbTiT_EZ
! unzip -qq test_set.zip

! mkdir test_set
! mv *.jpg test_set
# ! mv *.png test_set # if png present in test set

Downloading...
From: https://drive.google.com/u/0/uc?id=13S_GWX1f6K0ySqgeuQmH0pShhbTiT_EZ
To: /content/test_set.zip
181MB [00:01, 92.0MB/s]


In [ ]:
# Download all the models from Nilu's drive
#! gdown https://drive.google.com/u/0/uc?id=1VZea7fcT7wNlNxiJPunNtBVJBmkjCRUl # DenseNet201
#! gdown https://drive.google.com/u/0/uc?id=1t6DqQxwCB5Bwj_qlbzeImqqN2VXqzSDJ # InceptionResNetV2_train
#! gdown https://drive.google.com/u/0/uc?id=1cr_S8WfCdl1Pmx5b8GEkCBxbTRBrQjK9 # Xception
#! gdown https://drive.google.com/u/0/uc?id=1EgtqKPWP2XzXpGWc8fXz2V19sjT5ORVW # InceptionV3_train

In [ ]:
# From Jimut's drive
! gdown https://drive.google.com/u/0/uc?id=1NJjRRV-kwxB381O-8Gr8e6MC4IZWeJV0 # VGG16-best
! gdown https://drive.google.com/u/0/uc?id=1tmKkVBpJskNqFkQNh_CC4OTFRrzKG5Bg # InceptionV3_train
! gdown https://drive.google.com/u/0/uc?id=1kBtJvSWPnNrqEhhH9hwlXFXMWXeT_CyP # Xception
! gdown https://drive.google.com/u/0/uc?id=1GydERHlO1hKEzJhuA0MMlZ_xPKUAqSjG # InceptionResNetV2_train
! gdown https://drive.google.com/u/0/uc?id=1ZoHCSw3RaRTgMEnuaTASeYxsKnT8UMH_ # DenseNetV2

Downloading...
From: https://drive.google.com/u/0/uc?id=1NJjRRV-kwxB381O-8Gr8e6MC4IZWeJV0
To: /content/CXR_Covid-19_100e_VGG_16_all_500x500_1024_dropout-1024-3.h5
196MB [00:02, 74.3MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1tmKkVBpJskNqFkQNh_CC4OTFRrzKG5Bg
To: /content/CXR_Covid-19_100e_InceptionV3_500x500_1024-1024-3_val.h5
301MB [00:02, 138MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1kBtJvSWPnNrqEhhH9hwlXFXMWXeT_CyP
To: /content/CXR_Covid-19_50e_Xception_360x360_128-32-3.h5
254MB [00:01, 131MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1GydERHlO1hKEzJhuA0MMlZ_xPKUAqSjG
To: /content/CXR_Covid-19_50e_InceptionResNetV2_360x360_128-32-3_train.h5
657MB [00:06, 99.7MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1ZoHCSw3RaRTgMEnuaTASeYxsKnT8UMH_
To: /content/CXR_Covid-19_50e_DenseNet201_360x360_128-32-3.h5
224MB [00:01, 120MB/s] 


In [ ]:
from tensorflow.keras.utils import to_categorical
from PIL import Image

import pandas as pd
import sys
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import glob
import time
import cv2
import os
from tensorflow.keras.utils import to_categorical
from PIL import Image

import tensorflow as tf
from keras.regularizers import l2
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Conv2D, MaxPool2D,\
                                    GlobalMaxPool2D, Dropout, SpatialDropout2D, add, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import AUC, Precision, Recall, SensitivityAtSpecificity, PrecisionAtRecall, \
                                     TruePositives, TrueNegatives, FalsePositives, FalseNegatives


from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import keras

import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import glob
import time
import cv2
import os


In [ ]:
# Indices
index = {'normal': 0, 'covid': 1,  'pneumonia': 2}
rev_index = {0: 'normal',1: 'covid', 2: 'pneumonia'}

# For submission => 0:covid, 1: normal , 2: pneu
sub_index = {1:0, 0:1, 2:2}
# rev_sub_index = {0:1, 1:0, 2:2}

In [ ]:
from keras import backend as K
#from tensorflow import keras.applications.inception_resnet_v2

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())


from tensorflow import keras
model_incepV3 = keras.models.load_model('CXR_Covid-19_100e_InceptionV3_500x500_1024-1024-3_val.h5', custom_objects={'specificity': specificity, 'sensitivity': sensitivity})
model_denseNet = keras.models.load_model('CXR_Covid-19_50e_DenseNet201_360x360_128-32-3.h5', custom_objects={'specificity': specificity, 'sensitivity': sensitivity})
model_incepRNV2 = keras.models.load_model('CXR_Covid-19_50e_InceptionResNetV2_360x360_128-32-3_train.h5', custom_objects={'specificity': specificity, 'sensitivity': sensitivity})
model_Xcep = keras.models.load_model('CXR_Covid-19_50e_Xception_360x360_128-32-3.h5', custom_objects={'specificity': specificity, 'sensitivity': sensitivity})
model_VGG16 = keras.models.load_model('CXR_Covid-19_100e_VGG_16_all_500x500_1024_dropout-1024-3.h5', custom_objects={'specificity': specificity, 'sensitivity': sensitivity})

/usr/local/lib/python3.7/dist-packages/keras/layers/core.py:1045: UserWarning: tensorflow.python.keras.applications.inception_resnet_v2 is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


In [ ]:
##################################
H, W, C = 360, 360, 3
N_LABELS = len(index)
D = 1
##################################

def parse_filepath(filepath):
    try:
        #path, filename = os.path.split(filepath)
        label = filepath.split('/')[1]
        #filename, ext = os.path.splitext(filename)
        #label, _ = filename.split("_")
        return label
    except Exception as e:
        print('error to parse %s. %s' % (filepath, e))
        return None, None

files_validation = glob.glob("validation/*/*.*")
print("Total files valid = ",len(files_validation))

# create a pandas data frame of images, age, gender and race
attributes = list(map(parse_filepath, files_validation))

df_val = pd.DataFrame(attributes)
df_val['file'] = files_validation
df_val.columns = ['label', 'file']
df_val = df_val.dropna()
df_val.tail()

print(len(df_val))
p = np.random.permutation(len(df_val))
test_idx = p[:len(df_val)]

Total files valid =  3432
3432


In [ ]:
# Test on the train/val set to be sure...
from tensorflow.keras.utils import to_categorical
from PIL import Image

def get_data_generator(df, indices, for_training, batch_size=16):
    images, labels = [], []
    while True:
        # print("indices = ",indices)    
        # print("len indices = ",len(indices))
        for i in indices:
            r = df.iloc[i]
            # print(" r = ", r, " i = ",i)
            file, label = r['file'], r['label']
            # print("file, label = ",file, label)
            im_gray = Image.open(file).convert('L')
            # print("Shape = ",im_gray.shape)
            im_gray = im_gray.resize((360, 360))
            im = np.zeros(shape=(360, 360,3))
            
            im[:,:,0] = im_gray
            im[:,:,1] = im_gray
            im[:,:,2] = im_gray
            im = np.array(im) / 255.0

            # print(im.shape)
            images.append(im)
            # print(np.asarray([to_categorical(index[label], N_LABELS)]))
            # print(np.asarray([to_categorical(index[label], N_LABELS)]).shape)
            
            labels.append(to_categorical(index[label], N_LABELS))
            if len(images) >= batch_size:
                yield np.array(images), np.array(labels)
                images, labels = [], []
        # if not for_training:
        #     break

In [ ]:
test_gen = get_data_generator(df_val, test_idx, for_training=False)
dict(zip(model_Xcep.metrics_names, model_Xcep.evaluate(test_gen, steps=len(test_idx))))

  47/3432 [..............................] - ETA: 23:52 - loss: 0.3794 - accuracy: 0.9481 - sensitivity: 0.9481 - specificity: 0.9741

KeyboardInterrupt: ignored

In [ ]:
test_files = glob.glob('test_set/*.jpg')
print(len(test_files))
test_files = sorted(test_files)
test_files[:10]

1200


['test_set/0001.jpg',
 'test_set/0002.jpg',
 'test_set/0003.jpg',
 'test_set/0004.jpg',
 'test_set/0005.jpg',
 'test_set/0006.jpg',
 'test_set/0007.jpg',
 'test_set/0008.jpg',
 'test_set/0009.jpg',
 'test_set/0010.jpg']

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tqdm import tqdm
# y_pred_list = []
# y_test_list = []

sub_dic = {}
i=1

for file_path in tqdm(test_files):
    file_name = file_path.split('/')[1]
    # print(file_name)
    # break

    im_gray = Image.open(file_path).convert('L')
    im_gray_360 = im_gray.resize((360, 360))
    im_gray_500 = im_gray.resize((500, 500))
    im_360 = np.zeros(shape=(360, 360, 3))
    im_500 = np.zeros(shape=(500, 500, 3))

    im_360[:,:,0] = im_gray_360
    im_360[:,:,1] = im_gray_360
    im_360[:,:,2] = im_gray_360

    im_500[:,:,0] = im_gray_500
    im_500[:,:,1] = im_gray_500
    im_500[:,:,2] = im_gray_500

    im_360 = np.array(im_360) / 255.0

    im_500 = np.array(im_500) / 255.0

    y_pred_incepV3 = model_incepV3.predict(im_500[np.newaxis, ...])
    y_pred_denseNet = model_denseNet.predict(im_360[np.newaxis, ...])
    y_pred_incepRNV2 = model_incepRNV2.predict(im_360[np.newaxis, ...])
    y_pred_Xcep = model_Xcep.predict(im_360[np.newaxis, ...])
    y_pred_VGG = model_VGG16.predict(im_500[np.newaxis, ...])

    pred_incepV3 = sub_index[int(tf.math.argmax(y_pred_incepV3, axis=-1))]
    pred_denseNet = sub_index[int(tf.math.argmax(y_pred_denseNet, axis=-1))]
    pred_incepRNV2 = sub_index[int(tf.math.argmax(y_pred_incepRNV2, axis=-1))]
    pred_Xcep = sub_index[int(tf.math.argmax(y_pred_Xcep, axis=-1))]
    pred_VGG = sub_index[int(tf.math.argmax(y_pred_VGG, axis=-1))]

    '''
    print(pred_incepV3, "V3")
    print(pred_denseNet, "DN")
    print(pred_incepRNV2, "RNV2")
    print(pred_Xcep, "XCP")
    '''

    pred_dict = {0:0 , 1:0, 2:0}
    pred_dict[pred_incepV3] += 1
    pred_dict[pred_denseNet] +=1
    pred_dict[pred_incepRNV2] +=1
    pred_dict[pred_Xcep] +=1
    pred_dict[pred_VGG] +=1
    
    rev_pred_dict = {}
    for item in pred_dict:
      rev_pred_dict[item] = pred_dict[item]
    sub_dic[file_name] = max(pred_dict, key=pred_dict.get)
    '''
    i += 1
    if i==5:
      break
    '''

  0%|          | 0/1200 [00:00<?, ?it/s]

 90%|█████████ | 1081/1200 [11:23<01:14,  1.60it/s]

In [ ]:
sub_dic

{'0001.jpg': 2, '0002.jpg': 2, '0003.jpg': 1, '0004.jpg': 1}

In [ ]:
import csv
toCSV = []

for item in sub_dic:
    # print(item, " , ",sub_dic[item])
    toCSV.append({'case': item, 'class':sub_dic[item]})

keys = toCSV[0].keys()

with open('submission.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)